## 範例：估計隱蔽字，以及機率（信心）程度

In [1]:
# 安裝必要套件
!pip install transformers

In [2]:
# mask application
import torch
from transformers import BertTokenizer
from transformers import BertForMaskedLM    # 載入pretrained masked 語言模型並對有 [MASK] 的句子做預測


USE_PRETRAINED = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(USE_PRETRAINED)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [3]:
# 先只處理一個隱藏語詞
text1 = "[CLS]等到潮水[MASK]了，就知道誰沒穿褲子。"
text2 = "[CLS]數大[MASK]是美"
# 有兩個隱藏語詞
text3 = "[CLS]中華民[MASK]～中華[MASK]國～經得起考驗～～～" #"[CLS]中華[MASK][MASK]經得起考驗" # "[CLS]中華[MASK][MASK]萬萬稅"

text = text3
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens[:10], '...')
print(ids[:10], '...')

print('-=' * 120)



[CLS]中華民[MASK]～中華[MASK]國～經得起考驗～～～
['[CLS]', '中', '華', '民', '[MASK]', '～', '中', '華', '[MASK]', '國'] ...
[101, 704, 5836, 3696, 103, 8080, 704, 5836, 103, 1751] ...
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [4]:
# 除了 tokens 以外我們還需要辨別句子的 segment ids
tokens_tensor = torch.tensor([ids])  # (1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor)  # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(USE_PRETRAINED)
#clear_output()

# 使用 masked LM 估計 [MASK] 位置所代表的實際 token
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors) # 句子的內容和句子的位置
    predictions = outputs[0]
    print('預測出來的樣貌：', predictions.shape)
    print(predictions)


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


預測出來的樣貌： torch.Size([1, 17, 21128])
tensor([[[ -9.7597,  -9.8806,  -9.1634,  ...,  -9.1841,  -6.7692,  -6.2875],
         [-11.2953, -11.6374, -10.5959,  ...,  -9.9371,  -7.5620,  -6.1164],
         [-12.1168, -12.2003, -11.2903,  ..., -11.7408,  -8.4918,  -7.6806],
         ...,
         [-11.4153, -11.5151, -10.6668,  ..., -10.8767,  -7.4818,  -7.3177],
         [-11.3139, -11.3654, -10.4840,  ..., -11.0617,  -7.3490,  -7.2673],
         [-10.2792, -10.3030,  -9.5547,  ...,  -9.4556,  -6.3915,  -6.4515]]])


In [5]:
# 看看這個模型的架構
print(maskedLM_model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [6]:
# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
# 蓋牌處的 index
masked_index = 4    # 3  數大"便"是美    #5  等到潮水'退'了
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 當作預測值
print("輸入 tokens ：", tokens[:10], '...')
print('輸出', '-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    output_token = tokens.copy()
    output_token[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), output_token[:10]), '...')



輸入 tokens ： ['[CLS]', '中', '華', '民', '[MASK]', '～', '中', '華', '[MASK]', '國'] ...
輸出 --------------------------------------------------
Top 1 (25%)：['[CLS]', '中', '華', '民', '華', '～', '中', '華', '[MASK]', '國'] ...
Top 2 (17%)：['[CLS]', '中', '華', '民', '中', '～', '中', '華', '[MASK]', '國'] ...
Top 3 ( 4%)：['[CLS]', '中', '華', '民', '國', '～', '中', '華', '[MASK]', '國'] ...


In [7]:
# 如果有兩層 mask 呢？你會做嗎？
strlen_trim = 15

masked_index = 4    # 3  數大"便"是美    #5  等到潮水'退'了   # 中華民"國"
k1 = 5
topk = 1
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k1)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 當作預測值
print("輸入 tokens ：", tokens[:strlen_trim], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    output_token = tokens.copy()
    output_token[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), output_token[:strlen_trim]), '...')

print()
###### 這塊需不需要做? ######################################
# 假設在這裡，我們選了 top 1 ,也就是上一塊結果最強的字
output_token = tokens.copy()
output_token[masked_index] = predicted_tokens[2]

masked_index = 8    # 中華"民"國
k2 = 5
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k2)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。
print("輸入 tokens ：", output_token[:strlen_trim], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    output_token_2 = output_token.copy()
    output_token_2[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), output_token_2[:strlen_trim]), '...')

###### 這塊需不需要做? ######################################


輸入 tokens ： ['[CLS]', '中', '華', '民', '[MASK]', '～', '中', '華', '[MASK]', '國', '～', '經', '得', '起', '考'] ...
--------------------------------------------------
Top 1 (25%)：['[CLS]', '中', '華', '民', '華', '～', '中', '華', '[MASK]', '國', '～', '經', '得', '起', '考'] ...
Top 2 (17%)：['[CLS]', '中', '華', '民', '中', '～', '中', '華', '[MASK]', '國', '～', '經', '得', '起', '考'] ...
Top 3 ( 4%)：['[CLS]', '中', '華', '民', '國', '～', '中', '華', '[MASK]', '國', '～', '經', '得', '起', '考'] ...
Top 4 ( 1%)：['[CLS]', '中', '華', '民', '新', '～', '中', '華', '[MASK]', '國', '～', '經', '得', '起', '考'] ...
Top 5 ( 1%)：['[CLS]', '中', '華', '民', '大', '～', '中', '華', '[MASK]', '國', '～', '經', '得', '起', '考'] ...

輸入 tokens ： ['[CLS]', '中', '華', '民', '國', '～', '中', '華', '[MASK]', '國', '～', '經', '得', '起', '考'] ...
--------------------------------------------------
Top 1 (25%)：['[CLS]', '中', '華', '民', '國', '～', '中', '華', '華', '國', '～', '經', '得', '起', '考'] ...
Top 2 (17%)：['[CLS]', '中', '華', '民', '國', '～', '中', '華', '中', '國', '～', '經', '得', '起', '考

## Quiz - 1: 請自行設定一句話，並隱藏其中的一到數個字，使用預訓練模型得出最可能的前三名。請注意當有多個字詞被mask時，可能需要多層次解出字詞。（或用迴圈依序解出）

In [8]:
# 先只處理一個隱藏語詞
text1 = "[CLS]擁有夢想只是一種智力，實現夢想才是一種[MASK]力"
text2 = "[CLS]吃葡萄不[MASK]葡萄皮"
# 有兩個隱藏語詞
text3 = "[CLS]而我在這座城[MASK]遺[MASK]了你" #"[CLS]中華[MASK][MASK]經得起考驗" # "[CLS]中華[MASK][MASK]萬萬稅"

text = text2
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens[:10], '...')
print(ids[:10], '...')

print('-=' * 120)

# 除了 tokens 以外我們還需要辨別句子的 segment ids
tokens_tensor = torch.tensor([ids])  # (1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor)  # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(USE_PRETRAINED)
#clear_output()

# 使用 masked LM 估計 [MASK] 位置所代表的實際 token
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors) # 句子的內容和句子的位置
    predictions = outputs[0]
    print('預測出來的樣貌：', predictions.shape)
    print(predictions)

print('-=' * 120)

# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
# 蓋牌處的 index
masked_index = 5    # 3  數大"便"是美    #5  等到潮水'退'了
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 當作預測值
print("輸入 tokens ：", tokens[:10], '...')
print('輸出', '-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    output_token = tokens.copy()
    output_token[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), output_token[:10]), '...')


[CLS]吃葡萄不[MASK]葡萄皮
['[CLS]', '吃', '葡', '萄', '不', '[MASK]', '葡', '萄', '皮'] ...
[101, 1391, 5868, 5843, 679, 103, 5868, 5843, 4649] ...
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


預測出來的樣貌： torch.Size([1, 9, 21128])
tensor([[[-10.6811, -10.7025, -10.6890,  ..., -10.0228, -11.3664, -11.7372],
         [-10.7529, -10.7670, -10.7966,  ...,  -9.4612, -11.6503, -12.0201],
         [-11.6565, -11.7893, -11.6166,  ..., -10.6975, -12.6865, -13.0691],
         ...,
         [-11.5920, -11.7197, -11.5598,  ..., -10.7265, -12.5570, -13.0524],
         [-11.6638, -11.8489, -11.7338,  ..., -10.6922, -12.6182, -13.2668],
         [-11.2032, -11.1985, -11.1378,  ..., -10.5003, -12.4250, -12.2295]]])
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
輸入 tokens ： ['[CLS]', '吃', '葡', '萄', '不', '[MASK]', '葡', '萄', '皮'] ...
輸出 --------------------------------------------------
Top 1 (26%)：['[CLS]', '吃', '葡', '萄', '不', '，', '葡', '萄', '皮'] ...
Top 2 ( 3%)：['[CLS]', '吃', '葡', '萄', '不', '葡', '葡', '萄'

## **延伸思考：可不可能一次預測出不定數量的所有隱蔽字？**